In [49]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import us

In [9]:
mortality_path = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/outputs/mortality-rates-2021.npy'
mat = np.load(mortality_path)
boxplot_mat = mat.mean(axis=(0,2))

In [24]:
df = pd.DataFrame({'age': ['0-9', '10-49', '50-59', '60-69', '70-79', '80+'], 
                   'mortality': boxplot_mat})

In [25]:
fig = px.bar(df, x='age', y='mortality')
fig.update_layout(
        yaxis_title='Mortality rate, %',
        xaxis_title='Age group',
        font_size=16
    )

In [44]:
def load_and_clean_delphi_params(path: str) -> pd.DataFrame:
    # Load data from CSV file
    df = pd.read_csv(path)

    # Filter down to US only
    df = df[
        (df["Country"] == "US")
        & (df["Province"] != "None")
    ].drop(["Continent", "Country", "MAPE"], axis=1)

    # Rename columns
    df.rename(
        columns={
            "Province": "state",
            "Data Start Date": "start_date",
            "Median Day of Action": "intervention_time",
            "Rate of Action": "intervention_rate",
            "Infection Rate": "infection_rate",
            "Rate of Death": "death_rate",
            "Mortality Rate": "mortality_rate",
            "Rate of Mortality Rate Decay": "mortality_rate_decay",
            "Internal Parameter 1": "exposed_initial_param",
            "Internal Parameter 2": "infected_initial_param",
            "Jump Magnitude": "jump_magnitude",
            "Jump Time": "jump_time",
            "Jump Decay": "jump_decay"
        },
        inplace=True
    )

    # Cast start date as datetime object
    df["start_date"] = pd.to_datetime(df["start_date"])

    return df

In [45]:
df=load_and_clean_delphi_params('/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/delphi-parameters.csv')

In [66]:
new_df = df[['state', 'infection_rate']].reset_index(drop=True)
new_df['state'] = [us.states.lookup(x).abbr for x in new_df['state']]
new_df['infection_rate'] = np.minimum(new_df['infection_rate'].to_numpy(), 1)

In [74]:
import plotly.express as px

fig = px.choropleth(locations=new_df['state'],
                    locationmode="USA-states",
                    color=new_df['infection_rate'],
                    scope="usa", 
                    color_continuous_scale=px.colors.sequential.Reds, 
                    labels={'color':'Infection rate'})

fig.update_layout(font_size=17)